In [32]:
import pandas as pd
import numpy as np

pd.set_option('max_rows', 20)

DAILY_DATA_FOLDER = 'D:/analytics/stock/hist'
DAILY_SAMPLE_PATH = 'D:/analytics/stock/hist/002415.csv'

df = pd.read_csv(DAILY_SAMPLE_PATH, index_col=0, parse_dates=True, \
                 usecols=['date', 'close', 'p_change', 'ma5', 'ma10', 'ma20'], \
                 error_bad_lines=False)
df

,close,p_change,ma5,ma10,ma20
date,,,,,
2016-04-22,31.17,2.37,30.686,30.857,30.809
2016-04-21,30.45,0.79,30.732,30.806,30.803
2016-04-20,30.21,-1.60,30.880,30.831,30.875
2016-04-19,30.70,-0.65,31.064,30.946,30.953
2016-04-18,30.90,-1.59,31.004,31.016,31.038
2016-04-15,31.40,0.67,31.028,30.976,31.037
2016-04-14,31.19,0.19,30.880,30.918,30.967
2016-04-13,31.13,2.40,30.782,30.909,30.868
2016-04-12,30.40,-2.00,30.828,30.781,30.792


In [5]:
r = df.resample('W')
# daily average price change percent on weekly basis
r['p_change'].mean().dropna().sort_values()

date
2015-07-12   -5.413333
2015-06-21   -3.862000
2013-12-08   -2.440000
2015-03-08   -2.382500
2014-03-23   -2.336000
                ...   
2015-04-05    2.442000
2013-06-16    2.645000
2014-12-07    2.842000
2015-01-11    3.418000
2015-05-24    6.086000
Name: p_change, dtype: float64

In [2]:
import pandas as pd
import numpy as np

WEEKLY_DATA_FOLDER = 'E:/analytics/stock/hist-W-2016-0627'
WEEKLY_SAMPLE_PATH = 'E:/analytics/stock/hist-W-2016-0627/002643.csv'

df = pd.read_csv(WEEKLY_SAMPLE_PATH, index_col=0, parse_dates=True, \
                 usecols=['date', 'close', 'p_change', 'ma5', 'ma10', 'ma20'], \
                 error_bad_lines=False)
df

,close,p_change,ma5,ma10,ma20
date,,,,,
2016-06-27,58.10,3.11,52.744,50.356,42.310
2016-06-24,56.35,7.33,50.660,48.755,41.205
2016-06-17,52.50,6.56,49.130,47.041,40.087
2016-06-08,49.27,3.73,48.250,45.011,39.300
2016-06-03,47.50,-0.38,47.926,43.202,38.561
2016-05-27,47.68,-2.09,47.968,41.792,38.409
2016-05-20,48.70,1.25,46.850,40.089,38.051
2016-05-13,48.10,0.94,44.952,38.317,37.742
2016-05-06,47.65,-0.13,41.772,36.678,37.301


In [3]:
r_df = df.sort_index()
r_df.rolling(window=5)['close'].mean()

date
2016-01-08       NaN
2016-01-15       NaN
2016-01-22       NaN
2016-01-29       NaN
2016-02-05    37.140
2016-02-19    35.772
2016-02-26    35.592
2016-03-04    34.584
2016-03-11    33.980
2016-03-18    32.910
2016-03-25    32.068
2016-04-01    31.584
2016-04-08    31.682
2016-04-15    33.328
2016-04-22    35.616
2016-04-29    38.478
2016-05-06    41.772
2016-05-13    44.952
2016-05-20    46.850
2016-05-27    47.968
2016-06-03    47.926
2016-06-08    48.250
2016-06-17    49.130
2016-06-24    50.660
2016-06-27    52.744
Name: close, dtype: float64

In [18]:
s = df['ma5']
change_point = None
for i in range(0, len(s)-1):
    if s[i] < s[i+1]:
        change_point = i
        break
if not change_point:
    change_point = len(s) - 1
startdate = df.index[change_point]
increase = round(s[0] / s[change_point] - 1, 4) * 100
period = s[0:change_point+1]
print(startdate, increase, change_point+1, df[:change_point+1]['p_change'].mean(), \
      round(df[:change_point+1]['p_change'].std(), 2))

2016-06-03 00:00:00 10.05 5 4.07 3.07


In [84]:
import re
import glob
import os
import os.path
import pandas as pd
import sys

WEEKLY_DATA_FOLDER = 'D:/analytics/stock/hist-W-2016-0627'
DATE_FOR_PAUSE_CHECK = '2016-06-27'

reg = re.compile(r'(\d{6}).csv')
stocks = {t[1].group(1):t[0] for t in ((x, reg.search(x)) for x in glob.glob(WEEKLY_DATA_FOLDER + '/*.csv')) if t[1]}
# [os.path.isfile(x) for x in list(stocks.values())[:5]]

def find_increase_trend(path):
    df = pd.read_csv(path, index_col=0, parse_dates=True, \
                usecols=['date', 'close', 'p_change', 'ma5', 'ma10', 'ma20'], \
                error_bad_lines=False)
    if len(df) == 0:
        return None
    
    s = df['ma5']
    # the position from which MA starts to rise
    change_point = None
    for i in range(0, len(s)-1):
        if s[i] < s[i+1]:
            change_point = i
            break
    if change_point is None:
        change_point = len(s) - 1
    # if there is no increase in most recent period
    if change_point == 0:
        return None
    else:
        startdate = df.index[change_point]
        # use close price of two ends to calc total increase percent
        total_increase = round(df.iat[0, 0] / df.iat[change_point, 0] - 1, 4) * 100
        # the observation at change point doesn't count as increase, but only later ones
        trend_length = change_point
        # mean week-over-week increase percent over this period
        mean_increase = round(df[:change_point]['p_change'].mean(), 2)
        first_above_mean_position = df.index.get_loc(df[df['p_change'] >= mean_increase].index[0])
        # num of recent consecutive obserations whose WoW increase is lower than mean
        recent_below_mean_count = first_above_mean_position
        std = round(df[:change_point]['p_change'].std(), 2)
        pause = df.index[0] == pd.Timestamp(DATE_FOR_PAUSE_CHECK)
        return startdate, total_increase, trend_length, mean_increase, std, pause, recent_below_mean_count, 

resultmap = {}
for code in stocks:
    try:
#         print('processing ', code)
        res = find_increase_trend(stocks[code])
        ''' take as valid entry when following conditions are met:
                1. increasing trend lasts longer than 3 observations 
                2. actual increase percent over the period is above 10% (MA trails behind actual varation)
                3. increasing trend didn't considerably slowdown lately
        '''
        if res and res[2] >= 3 and res[1] > 10 and res[-1] <= 1:
            resultmap[code] = res
    except Exception as ex:
        print('error occurred in processing %s: %s' % (code, ex))
df = pd.DataFrame.from_dict(resultmap, orient='index')
df.columns = ['startdate', 'increase', 'length', 'mean', 'std', 'pause', 'RSL']
df

,startdate,increase,length,mean,std,pause,RSL
603377,2016-05-20,22.91,6,3.66,6.32,True,0
002500,2016-03-11,66.20,16,3.50,7.87,True,1
002049,2016-05-27,25.86,5,4.92,7.58,True,0
603118,2016-05-20,19.93,6,3.22,5.95,True,1
002553,2016-06-08,11.48,3,3.72,3.15,True,0
603021,2016-06-08,18.84,3,5.92,0.78,True,1
600139,2016-03-11,41.24,4,9.12,5.65,False,0
002178,2016-06-08,18.07,3,6.11,11.68,True,1
600846,2016-05-27,12.87,5,2.61,6.26,True,0
002403,2016-02-26,50.55,17,2.53,4.59,True,0


In [35]:
OUTPUT_PATH = 'D:/analytics/stock/analysis_weekly.csv'
df.to_csv(OUTPUT_PATH)

In [53]:
find_increase_trend(WEEKLY_DATA_FOLDER + '/002176.csv')

(Timestamp('2016-05-13 00:00:00'),
 34.229999999999997,
 7,
 4.46,
 6.4100000000000001,
 4)

In [76]:
import numpy as np

pd.set_option('max_rows', 20)

BASIC_DATA_PATH = r'D:\analytics\stock\basic.csv'
basic_df = pd.read_csv(BASIC_DATA_PATH, index_col=False, dtype={'code':np.str}, \
                       usecols=['code', 'pe', 'pb', 'outstanding', 'totals', 'esp', 'timeToMarket'], \
                       error_bad_lines=False)
basic_df.set_index('code', inplace=True)
basic_df['timeToMarket'] = pd.to_datetime(basic_df['timeToMarket'], errors='coerce', format='%Y%m%d')
basic_df = basic_df[basic_df['timeToMarket'].notnull()]
basic_df
# basic_df['timeToMarket'].convert_objects(convert_dates='coerce')
# print(basic_df.dtypes)
# basic_df
# # basic_df[basic_df['timeToMarket']=='0']

,pe,outstanding,totals,esp,pb,timeToMarket
code,,,,,,
603958,21.30,5436.00,21736.00,0.155,3.19,2016-06-29
002802,30.34,2700.00,10800.00,0.113,3.78,2016-06-29
600069,0.00,82537.40,124910.29,-0.035,3.10,1997-04-30
002325,34.06,91356.09,120185.09,0.068,3.34,2009-12-22
002651,100.32,31280.41,100250.00,0.025,5.29,2012-01-06
300074,0.00,43451.62,52800.00,-0.017,5.79,2010-04-27
002666,52.27,44626.83,75432.92,0.045,2.66,2012-03-27
600767,0.00,34091.01,34101.01,-0.052,18.54,1996-11-15
600528,65.12,145920.00,145920.00,0.043,2.63,2001-05-28


In [60]:
REPORT_DATA_FOLDER = r'D:/analytics/stock/report'

reg = re.compile(r'(\d{4}-\d).csv')
reports = {t[1].group(1):t[0] for t in ((x, reg.search(x)) for x in glob.glob(REPORT_DATA_FOLDER + '/*.csv')) if t[1]}
report_dfs = []
report_terms = []
for term in reports:
    rdf = pd.read_csv(reports[term], index_col=False, dtype={'code':np.str}, \
                           usecols=['code', 'roe', 'profits_yoy'], \
                           error_bad_lines=False)
    rdf.set_index('code', inplace=True)
    d = rdf.index.duplicated()
#     print('duplicates in %s: %d (%s)' % (term, len(d[d==True]), rdf.index[d==True][:5]))
    rdf.drop_duplicates(inplace=True)
    report_dfs.append(rdf)
    report_terms.append(term)
# print(report_dfs)
all_report_df = pd.concat(report_dfs, keys=report_terms, axis=1, join='outer')
all_report_df

2015-4             2015-3             2015-2             2016-1  \
          roe profits_yoy    roe profits_yoy    roe profits_yoy    roe   
000001  14.94       10.42  12.39       13.04    NaN         NaN   3.70   
000002  19.14       15.08   7.69        6.14   5.41        0.77   0.83   
000004   1.54      -67.19   2.92     -142.69   1.50     -125.76    NaN   
000005  -8.07     -227.98  -4.56       -6.61  -3.52       -8.05  -0.59   
000006   9.77      -17.97   6.56       15.54    NaN         NaN   1.11   
000007   3.87     -146.35  -3.35       74.22  -2.12       89.72   2.05   
000008   7.60     2290.28    NaN         NaN   2.93      710.43   0.24   
000009  19.70      160.12  17.55      160.90   3.38      -48.53    NaN   
000010   4.05     -144.86   0.50     -120.03   0.29     -125.35  -1.51   
000011   7.59      -62.44    NaN         NaN   1.52      -72.19    NaN   
...       ...         ...    ...         ...    ...         ...    ...   
603968  12.34        4.24  11.43        1.05  10.41       23.49   1.74   
603969   9.51       -2.22    NaN         NaN   5.57       -6.31    NaN   
603979   7.70      -27.79   7.94      -13.87   6.84       -9.71   1.23   
603988   6.89      -21.29   5.74      -16.53   3.82      -22.99   1.81   
603989  16.52       25.14  12.12       11.20   9.36      -13.29    NaN   
603993   4.77      -58.28    NaN         NaN   3.15      -53.92   0.81   
603996  13.18        5.15    NaN         NaN   6.19         NaN   0.19   
603997    NaN         NaN  11.55      -12.44   8.32      -11.73   4.36   
603998  10.75       11.96   8.62       15.08    NaN         NaN    NaN   
603999   9.86      -28.07    NaN         NaN   4.20         NaN   0.74   

                    
       profits_yoy  
000001        8.12  
000002       28.14  
000004         NaN  
000005      -15.65  
000006      -75.79  
000007     -257.50  
000008      -79.46  
000009         NaN  
000010     1407.81  
000011         NaN  
...            ...  
603968      -47.32  
603969         NaN  
603979       -4.54  
603988      -22.82  
603989         NaN  
603993      -53.67  
603996      -75.55  
603997       13.74  
603998         NaN  
603999      -20.19  

[2864 rows x 8 columns]

In [85]:
CONSOLIDATED_DATA_PATH = r'D:/analytics/stock/consolidated.csv'
consolidated = df.join(basic_df).join(all_report_df)
consolidated.to_csv(CONSOLIDATED_DATA_PATH)